In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import sys, os
sys.path.append('/'.join(os.getcwd().split('/')[:4]))
from config.get import cfg

2022-01-11 14:50:45.511767: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2


In [2]:
import pandas_ta as ta

In [3]:
data = pd.read_csv(cfg['files']['liquid']['preprocessed_data'],nrows=10_000_000).drop(columns=['Unnamed: 0', 'baseAmount','quoteAmount', 'gasValue','time'])

In [4]:
data.head()

,cycle_id,token1,token2,quotePrice,gasPrice
0,1002,WETH,BUIDL,22550.438239,20.000000
1,1002,WETH,BUIDL,9462.858024,101.000004
2,1002,WETH,BUIDL,32966.161101,30.000001
3,1002,WETH,BUIDL,20101.046750,31.000001
4,1002,WETH,BUIDL,14104.537180,31.000300


# Rule based indicators

> **Note**: we tried to use the ta.Strategy but it was too slow on a Grouper object

The following indicators are builts using two windows(5 and 20) on both prices and log returns:

1. SMA
5. volatily
  


In [5]:
def logret(data, cols = ['quotePrice', 'gasPrice']):
    """Constructs log return series for each col in cols per token pair data"""
    fcols = []
    for col in cols:
        ncol = f'logret_{col}'
        data[ncol] = np.log(data[col])
        fcols.append(ncol)
    
    grouped = data.groupby(['cycle_id', 'token1','token2'])[fcols]
    log_ret = grouped.diff()
    data.drop(columns=fcols,inplace=True)
    return log_ret

In [6]:
logret_data = logret(data)

/ssoft/spack/arvine/v1/opt/spack/linux-rhel7-skylake_avx512/gcc-8.4.0/python-3.7.7-drpdlwdbo3lmtkcbckq227ypnzno4ek3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
for c in logret_data.columns:
    data[c] = logret_data[c]

In [8]:
data.head()

,cycle_id,token1,token2,quotePrice,gasPrice,logret_quotePrice,logret_gasPrice
0,1002,WETH,BUIDL,22550.438239,20.000000,NaN,NaN
1,1002,WETH,BUIDL,9462.858024,101.000004,-0.868380,1.619388
2,1002,WETH,BUIDL,32966.161101,30.000001,1.248107,-1.213923
3,1002,WETH,BUIDL,20101.046750,31.000001,-0.494710,0.032790
4,1002,WETH,BUIDL,14104.537180,31.000300,-0.354275,0.000010


In [9]:
def get_ta_data(window):
    g = data.groupby(['cycle_id', 'token1','token2']).rolling(window).agg({
                     'quotePrice'        : ['mean','std'],
                     'gasPrice'          : ['mean','std'],
                     'logret_quotePrice' : ['mean','std'],
                     'logret_gasPrice'   : ['mean','std'], 
            })
    new_cols = [f'quotePrice_{window}'       ,f'gasPrice_{window}',
                f'logret_quotePrice_{window}',f'logret_gasPrice_{window}' ]
    g.columns.set_levels(new_cols,level=0,inplace=True)
    return g.fillna(0.0)

In [10]:
ta_data_20 = get_ta_data(20)

In [11]:
ta_data_5 = get_ta_data(5)

In [12]:
ruled_based_embedding = pd.concat((ta_data_5, ta_data_20),axis=1)
ruled_based_embedding.columns = ["_".join(a) for a in ruled_based_embedding.columns.to_flat_index()]
ruled_based_embedding.head()

quotePrice_5_mean  quotePrice_5_std  \
cycle_id token1 token2                                            
1002     BUIDL  STA    600           0.000000          0.000000   
                       601           0.000000          0.000000   
                       602           0.000000          0.000000   
                       603           0.000000          0.000000   
                       604           8.742736          3.170428   

                            gasPrice_5_mean  gasPrice_5_std  \
cycle_id token1 token2                                        
1002     BUIDL  STA    600         0.000000        0.000000   
                       601         0.000000        0.000000   
                       602         0.000000        0.000000   
                       603         0.000000        0.000000   
                       604        46.423853       13.554863   

                            logret_quotePrice_5_mean  logret_quotePrice_5_std  \
cycle_id token1 token2                                                          
1002     BUIDL  STA    600                       0.0                      0.0   
                       601                       0.0                      0.0   
                       602                       0.0                      0.0   
                       603                       0.0                      0.0   
                       604                       0.0                      0.0   

                            logret_gasPrice_5_mean  logret_gasPrice_5_std  \
cycle_id token1 token2                                                      
1002     BUIDL  STA    600                     0.0                    0.0   
                       601                     0.0                    0.0   
                       602                     0.0                    0.0   
                       603                     0.0                    0.0   
                       604                     0.0                    0.0   

                            quotePrice_20_mean  quotePrice_20_std  \
cycle_id token1 token2                                              
1002     BUIDL  STA    600                 0.0                0.0   
                       601                 0.0                0.0   
                       602                 0.0                0.0   
                       603                 0.0                0.0   
                       604                 0.0                0.0   

                            gasPrice_20_mean  gasPrice_20_std  \
cycle_id token1 token2                                          
1002     BUIDL  STA    600               0.0              0.0   
                       601               0.0              0.0   
                       602               0.0              0.0   
                       603               0.0              0.0   
                       604               0.0              0.0   

                            logret_quotePrice_20_mean  \
cycle_id token1 token2                                  
1002     BUIDL  STA    600                        0.0   
                       601                        0.0   
                       602                        0.0   
                       603                        0.0   
                       604                        0.0   

                            logret_quotePrice_20_std  logret_gasPrice_20_mean  \
cycle_id token1 token2                                                          
1002     BUIDL  STA    600                       0.0                      0.0   
                       601                       0.0                      0.0   
                       602                       0.0                      0.0   
                       603                       0.0                      0.0   
                       604                       0.0                      0.0   

                            logret_gasPrice_20_std  
cycle_id token1 token2                         

In [13]:
import gc
del ta_data_20
del ta_data_5
del data
gc.collect()

42

In [15]:
ruled_based_embedding.reset_index().to_csv(cfg['files']['liquid']['ruled_based']['raw_features'])